In [80]:
#Parameters
samplesize = 10000

In [81]:
import pandas as pd
df_mv = pd.read_csv(r"C:\Users\sjlal\NYP\Project\FinalData\movies_metadata.csv")
df_mv1 = df_mv[['id','genres','original_title', 'title']]
df_mv1.drop_duplicates(subset = ['title'], keep = 'first', inplace = True)
df_mv2 = df_mv1.sample(n=samplesize, replace=False, random_state=42)
df_rating = pd.read_csv(r"C:\Users\sjlal\NYP\Project\FinalData\ratings.csv")
# generate mv features
df_mv['id'] = df_mv['id'].astype(str)
df_rating['movieId'] = df_rating['movieId'].astype(str)
df_rate_mv_pro = pd.merge(df_mv, df_rating, how='right', left_on=['id'], right_on=['movieId'])
df_rate_mv_pro1 = df_rate_mv_pro[['genres','id','title','userId','movieId', 'rating']]
df_rate_mv_pro1.dropna(inplace=True)

<ipython-input-81-cbb428eb33a9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mv1.drop_duplicates(subset = ['title'], keep = 'first', inplace = True)
<ipython-input-81-cbb428eb33a9>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rate_mv_pro1.dropna(inplace=True)


In [92]:
import ast
def get_genre(outterlist):
    outterlist = ast.literal_eval(outterlist)
    lst = []
    for a in outterlist:
        try:
            lst.append(a['name'])
        except IndexError:
            lst.append('')
    return '|'.join(lst)

df_rate_mv_pro1['genres_descp'] = df_rate_mv_pro1['genres'].apply(get_genre)
df_mv3 = df_rate_mv_pro1.loc[~df_rate_mv_pro1["id"].str.contains('-')]
df_mv3.head()

<ipython-input-92-5f599f434c99>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rate_mv_pro1['genres_descp'] = df_rate_mv_pro1['genres'].apply(get_genre)


,genres,id,title,userId,movieId,rating,genres_descp
0,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...",110,Three Colors: Red,1,110,1.0,Drama|Mystery|Romance
1,"[{'id': 18, 'name': 'Drama'}]",147,The 400 Blows,1,147,4.5,Drama
2,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",858,Sleepless in Seattle,1,858,5.0,Comedy|Drama|Romance
4,"[{'id': 18, 'name': 'Drama'}]",1246,Rocky Balboa,1,1246,5.0,Drama
5,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",1968,Fools Rush In,1,1968,4.0,Drama|Comedy|Romance


In [93]:

df_rating1 = df_rating.sample(n=samplesize, replace=False, random_state=1)
R = pd.pivot_table(df_rating1, values='rating', index=['userId'],columns=['movieId'], fill_value=0).to_numpy()
# R = df_rating_mv[['userId','rating']].values
print ('{0}x{1} user by user matrix'.format(*R.shape))
Rrow = R.shape[0]
Rrowa = int(Rrow*0.8)
Rcol = R.shape[1]
Rcola = int(Rcol*0.8)

9145x3525 user by user matrix


In [116]:
df_mv3[df_mv3.duplicated(subset =['id', 'title']) == True]

,genres,id,title,userId,movieId,rating,genres_descp
52,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",858,Sleepless in Seattle,3,858,4.0,Comedy|Drama|Romance
56,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",1968,Fools Rush In,3,1968,3.0,Drama|Comedy|Romance
61,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",648,Beauty and the Beast,4,648,4.0,Drama|Fantasy|Romance
89,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",2762,Young and Innocent,4,2762,5.0,Drama|Crime
116,"[{'id': 35, 'name': 'Comedy'}]",4226,Shriek If You Know What I Did Last Friday the ...,4,4226,3.0,Comedy
...,...,...,...,...,...,...,...
26025338,"[{'id': 10769, 'name': 'Foreign'}]",48780,Boat,270896,48780,5.0,Foreign
26025340,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",49530,In Time,270896,49530,4.0,Action|Thriller|Science Fiction
26025347,"[{'id': 18, 'name': 'Drama'}]",54001,The Traveler,270896,54001,4.0,Drama
26025349,"[{'id': 28, 'name': 'Action'}, {'id': 10769, '...",54503,The Mystery of Chess Boxing,270896,54503,4.0,Action|Foreign


In [84]:
df_mv3.drop_duplicates(subset =['id', 'title'], keep = 'first', inplace = True)
df_mv3 = df_mv3.loc[df_mv3['genres_descp'].str.len()>3]
df_mv4 = df_mv3.sample(n=samplesize, replace=False, random_state=1)
df_mv4.reset_index(inplace=True)
df_mv_fea_frame = pd.pivot_table(df_mv4, values='rating', index=['movieId'],columns=['userId'], fill_value=0)
df_mv_fea = pd.pivot_table(df_mv4, values='rating', index=['movieId'],columns=['userId'], fill_value=0).to_numpy()

print ('{0}x{1} user by movie matrix'.format(*df_mv_fea.shape))
Fearow = df_mv_fea.shape[0]
Fearowa = int(Fearow*0.8)

Feacol = df_mv_fea.shape[1]
Feacola = int(Feacol*0.8)

ValueError: Cannot take a larger sample than population when 'replace=False'

In [ ]:

from sklearn.decomposition import TruncatedSVD
# 80% train data
train_user = R[:Rrowa, :Rcola]
test_user = R[Rrowa:, Rcola:]
train_mv = df_mv_fea[:Fearowa, :Feacola]
test_mv = df_mv_fea[Fearowa:, Feacola:]

movie_svd = TruncatedSVD(n_components=2)
movie_features = movie_svd.fit_transform(train_mv)

print ("movie_features.shape = {0}".format(movie_features.shape))


user_svd = TruncatedSVD(n_components=10)
user_features = user_svd.fit_transform(train_user)

print ("user_features.shape = {0}".format(user_features.shape))

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

tsne = TSNE(perplexity=5, n_components=2, init="pca", n_iter=5000)
plot_only = 50
coords = tsne.fit_transform(movie_features[:plot_only, :])

plt.figure(figsize=(18, 18))
labels = [df_mv4.iloc[i].title for i in range(plot_only)]
for i, label in enumerate(labels):
    x, y = coords[i, :]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(10, 4),
                 textcoords="offset points",
                 ha="right",
                 va="bottom")

plt.show()

In [ ]:
df_rating_avg = df_rating.groupby(['movieId'])['rating'].mean().reset_index()
df_rating_avg['rating'] = df_rating_avg['rating'].round(1)
df_rating_avg['movieId'] = df_rating_avg['movieId'].astype(str)
df_rating2 = df_rating_avg.set_index('movieId')
df_train = pd.merge(df_mv_fea_frame,df_rating2, left_index=True, right_index=True, how='right')
df_train.dropna(inplace=True)
import numpy as np
targets = np.array(df_train.rating)
data = np.array(df_train.drop(['rating'], axis = 1))

print ("targets.shape = {0}".format(targets.shape))
print ("data.shape = {0}".format(data.shape))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error 
import math

regressor = GradientBoostingRegressor(learning_rate=0.1, n_estimators=200, verbose=1)
regressor.fit(data, targets)

print ("RSME= ", math.sqrt(mean_squared_error( regressor.predict(data), targets )))
print ("Sample Size = ", samplesize)